In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
pd.set_option('display.max_columns', None)

In [ ]:
# Load data and display basic information
loan_data = pd.read_csv('../data/loan_data.csv')
loan_data.info()

In [ ]:
# Analyze 'loan_status' (our target variable)

loan_status_counts = loan_data['loan_status'].value_counts()
loan_status_percentages = loan_data['loan_status'].value_counts(normalize=True)
loan_status_df = pd.concat([loan_status_counts, loan_status_percentages], axis=1)
loan_status_df

Notice that the following columns are filled entirely with null values and should be dropped: wtd_loans, interest_rate, num_rate, numrate. There appear to be a large number of sparsely-populated records with 9524 non-nulls. These 476 records only contain loan_amnt, funded_amnt, and addr_state. These records do not appear to have a significantly different loan amount size or state distribution than the non-null data, so we are fine dropping them.

In [ ]:
df1 = loan_data.loc[loan_data['total_pymnt'].isna(), ['loan_amnt', 'funded_amnt']].describe()
df2 = loan_data.loc[~loan_data['total_pymnt'].isna(), ['loan_amnt', 'funded_amnt']].describe()

combined_df = pd.concat([df1, df2], axis=1, keys=['total_pymnt is NaN', 'total_pymnt is not NaN'])
combined_df

In [ ]:
df1 = loan_data.loc[loan_data['total_pymnt'].isna(), 'addr_state'].value_counts(normalize=True)
df2 = loan_data.loc[~loan_data['total_pymnt'].isna(), 'addr_state'].value_counts(normalize=True)

combined_df = pd.concat([df1, df2], axis=1, keys=['total_pymnt is NaN', 'total_pymnt is not NaN'])
combined_df.head(10)

In [ ]:
# Generate profiling report for additional analysis and alerts
profile = ProfileReport(loan_data, title="Loan Data Report")
profile.to_file("../analysis_outputs/profile_report.html")
profile

Necessary data pre-processing:

    1. Drop unnecessary columns
    2. Drop rows where only 'loan_amnt', 'funded_amnt', and 'addr_state' are non-null
    3. Drop the 'In-Grace' period records
    4. Combine loan statuses into "good" and "bad" categories
    5. Convert the employment length to numeric values
    6. Handle missing values (NaNs) for no employment history
    7. Fill NaNs with zeroes in 'mths_since_last_delinq'
    8. Replace '36 months' with 0 and '60 months' with numeric values
    9. Group rare categories into 'OTHER'
    10. Convert 'int_rate2' to float 